In [1]:
import pandas as pd
import os
import sqlalchemy as db
import numpy as np

Connection to DLSU Cloud DB (PAL)
- `connection.json` is abstracted for data privacy

In [4]:
conn = pd.read_json('connection.json', typ='series')
host = conn['host']
user = conn['user']
password = conn['password']
port = conn['port']

db_name = input('DB Name: ')
engine = db.create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + ':' + port + '/' + db_name)

DB Name: Metadata


# Aircraft Details

In [5]:
query = "SELECT * FROM Aircrafts;"
ac = pd.read_sql(query, engine)

In [6]:
ac

,PR/2P,PAL/GAP,Acft Type,Acft Short,Long Reg,Short Reg,Group Equipment
0,PR,PAL,B777,B77,RPC7772,772,B777
1,PR,PAL,B777,B77,RPC7773,773,B777
2,PR,PAL,B777,B77,RPC7774,774,B777
3,PR,PAL,B777,B77,RPC7775,775,B777
4,PR,PAL,B777,B77,RPC7776,776,B777
...,...,...,...,...,...,...,...
96,2P,GAP,DH4,DH4,RPC3036,36,Q400
97,2P,GAP,DH3,DH3,RPC3016,16,Q300
98,2P,GAP,DH3,DH3,RPC3017,17,Q300
99,2P,GAP,DH3,DH3,RPC3018,18,Q300


In [7]:
palReg = ac[ac['PAL/GAP'] == 'PAL']
gapReg = ac[ac['PAL/GAP'] == 'GAP']

In [8]:
palTypes = palReg['Acft Type'].unique()
gapTypes = gapReg['Acft Type'].unique()

print('PAL list: ', palTypes)
print('PAL type count: ', len(palTypes))
print('GAP list: ', gapTypes)
print('GAP type count: ', len(gapTypes))

ac[ac['PAL/GAP'] == 'PAL'][['Acft Type', 'Long Reg', 'Short Reg']].groupby('Acft Type').count()

PAL list:  ['B777' 'A350' 'A332' 'A333' 'A321C' 'A321NX' 'A321N' 'A320']
PAL type count:  8
GAP list:  ['A321C' 'A320' 'DH8' 'DH4' 'DH3']
GAP type count:  5


,Long Reg,Short Reg
Acft Type,,
A320,5,5
A321C,18,18
A321N,6,6
A321NX,2,2
A332,7,7
A333,8,8
A350,6,6
B777,10,10


In [9]:
palRegDict = {}
gapRegDict = {}

for each in list(palTypes):
    palRegDict[each] = list(palReg[palReg['Acft Type'] == each]['Long Reg'])

for each in list(gapTypes):
    gapRegDict[each] = list(gapReg[gapReg['Acft Type'] == each]['Long Reg'])

# Population Count

In [10]:
# DETERMINE DATA POINTS FOR EACH MONTH
# PAL and GAP
count = pd.read_excel('count.xlsx')
count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70237 entries, 0 to 70236
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             70237 non-null  datetime64[ns]
 1   fltno            70232 non-null  object        
 2   Long Reg         70237 non-null  object        
 3   iter_date        70237 non-null  int64         
 4   PR/2P            70237 non-null  object        
 5   PAL/GAP          70237 non-null  object        
 6   Acft Type        70237 non-null  object        
 7   Group Equipment  70237 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 4.3+ MB


In [11]:
palcount = count[count['PAL/GAP'] == 'PAL'].reset_index().drop('index', axis=1)
gapcount = count[count['PAL/GAP'] == 'GAP'].reset_index().drop('index', axis=1)
print("pal count:", len(palcount))
print("GAP count:", len(gapcount))
print("TOTAL    :", len(count))

pal count: 36474
GAP count: 33763
TOTAL    : 70237


In [15]:
count[(count['PAL/GAP'] == 'PAL') & (count['Acft Type'] == 'B777')]

,date,fltno,Long Reg,iter_date,PR/2P,PAL/GAP,Acft Type,Group Equipment
60,2022-01-01,PAL102,RPC7776,0,PR,PAL,B777,B777
61,2022-01-01,PAL103,RPC7782,0,PR,PAL,B777,B777
62,2022-01-01,PAL105,RPC7773,0,PR,PAL,B777,B777
99,2022-01-01,PAL5104,RPC7772,0,PR,PAL,B777,B777
105,2022-01-01,PAL592,RPC7772,0,PR,PAL,B777,B777
...,...,...,...,...,...,...,...,...
70232,2023-09-20,PAL113,RPC7772,0,PR,PAL,B777,B777
70233,2023-09-21,PAL102,RPC7772,0,PR,PAL,B777,B777
70234,2023-09-22,PAL103,RPC7772,0,PR,PAL,B777,B777
70235,2023-09-23,PAL116,RPC7772,0,PR,PAL,B777,B777


In [13]:
count[(count['PAL/GAP'] == 'GAP') & (count['Acft Type'] == 'A321C')]

,date,fltno,Long Reg,iter_date,PR/2P,PAL/GAP,Acft Type,Group Equipment
8,2022-01-01,GAP2129,RPC9927,0,2P,GAP,A321C,ANB
9,2022-01-01,GAP2130,RPC9927,0,2P,GAP,A321C,ANB
10,2022-01-01,GAP2131,RPC9921,0,2P,GAP,A321C,ANB
11,2022-01-01,GAP2132,RPC9921,0,2P,GAP,A321C,ANB
12,2022-01-01,GAP2135,RPC9921,0,2P,GAP,A321C,ANB
...,...,...,...,...,...,...,...,...
69397,2022-12-31,GAP2808,RPC9924,0,2P,GAP,A321C,ANB
69398,2022-12-31,GAP2823,RPC9927,0,2P,GAP,A321C,ANB
69399,2022-12-31,GAP2824,RPC9927,0,2P,GAP,A321C,ANB
69400,2022-12-31,GAP2825,RPC9924,0,2P,GAP,A321C,ANB
